In [ ]:
# 3. Import libraries and modules
!pip install git+https://github.com/qubvel/classification_models.git
from classification_models.keras import Classifiers
import numpy as np
np.random.seed(123)  # for reproducibility
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Embedding
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
!pip install adabelief-tf==0.1.0
from adabelief_tf import AdaBeliefOptimizer
!pip install keras-rectified-adam
from keras_radam.training import RAdamOptimizer
!pip install keras-adabound
from keras_adabound import AdaBound
!pip install git+https://github.com/tensorflow/addons
import tensorflow_addons





In [ ]:
import IPython.display as ipd
import librosa
import os
import librosa.display

df = pd.read_csv("../input/urbansound8k/UrbanSound8K.csv")
df['class'].value_counts()
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

extracted_features=[]
for i in range(8732):
    file_name = '../input/urbansound8k/fold' + str(df["fold"][i]) + '/' + df["slice_file_name"][i]
    final_class_labels=df["class"][i]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])
    
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

y=np.array(pd.get_dummies(y))

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=0,stratify=y)
num_labels=y.shape[1]


In [ ]:
X_train = X_train.reshape(7422,1, 40)
print(X_train.shape)
print(X_train)

In [ ]:
y_train = y_train.reshape(7422,1, 10)
print(y_train.shape)
print(y_train)

In [ ]:
X_test = X_test.reshape(1310,1, 40)
print(X_test.shape)
print(X_test)

In [ ]:
y_test = y_test.reshape(1310,1,10)
print(y_test.shape)
print(y_test)

In [ ]:
from keras.layers import LSTM,Input

model = Sequential()
model.add(LSTM(40,input_shape=(1,40), return_sequences=True))
model.add(LSTM(40,input_shape=(1,40),return_sequences=True))
model.add(Dense(512,activation ='relu'))
model.add(Dropout(0.2))
model.add(Dense(10,activation ='softmax'))
model.summary()

In [ ]:
k = []
ok = 0
yes = 0
zet = 0
yep = 0


opt =  [keras.optimizers.Adam(learning_rate=0.001),
                  keras.optimizers.SGD(learning_rate=0.01),
                  keras.optimizers.Nadam(learning_rate=0.001),
                  keras.optimizers.RMSprop(learning_rate=0.001),
                  AdaBeliefOptimizer(learning_rate=0.001),
                  RAdamOptimizer(learning_rate=0.001),
                  AdaBound(learning_rate=0.001, final_lr=0.1),
                  tensorflow_addons.optimizers.yogi.Yogi(learning_rate=0.001)]
        
for t in opt:
    del model
    #del base_model
    #del x
    #del output
    if (t == opt[0]):
        print("-------------------------------------optimizer = Adam-------------------------------------")
    elif (t == opt[1]):
            print("-------------------------------------optimizer = SGD-------------------------------------")
    elif (t == opt[2]):
            print("-------------------------------------optimizer = Nadam-------------------------------------")
    elif (t == opt[3]):
            print("-------------------------------------optimizer = RMSprop-------------------------------------")
    elif (t == opt[4]):
            print("-------------------------------------optimizer = AdaBelief-------------------------------------")
    elif (t == opt[5]):
            print("-------------------------------------optimizer = RAdam-------------------------------------")

    elif (t == opt[6]):
            print("-------------------------------------optimizer = AdaBound-------------------------------------")

    elif (t == opt[7]):
            print("-------------------------------------optimizer = Yogi-------------------------------------")

    model = Sequential()
    model.add(LSTM(40,input_shape=(1,40), return_sequences=True))
    model.add(LSTM(40,input_shape=(1,40),return_sequences=True))
    model.add(Dense(512,activation ='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation ='softmax'))
    # 8. Compile model
    model.compile(loss='categorical_crossentropy',
              optimizer = t,
              metrics=['accuracy'])


       # 9. Fit model on training data
    k.append(model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test),verbose=1))



while (ok<len(k)):  



    plt.figure(figsize=(10,10))
    plt.plot(k[ok+0].history['accuracy'],color='b', label="opt = Adam")
    plt.plot(k[ok+1].history['accuracy'],color='r', label="opt = SGD")
    plt.plot(k[ok+2].history['accuracy'],color='g', label="opt = Nadam")
    plt.plot(k[ok+3].history['accuracy'],color='c', label="opt = RMSprop")
    plt.plot(k[ok+4].history['accuracy'],color='m', label="opt = AdaBelief")
    plt.plot(k[ok+5].history['accuracy'],color='k', label="opt = RAdam")
    plt.plot(k[ok+6].history['accuracy'],color='y', label="opt = AdaBound")
    plt.plot(k[ok+7].history['accuracy'],color='#A18820', label="opt = Yogi")
    
    plt.xlabel('Epochs')
    plt.ylabel('accuracy')
    plt.legend()
    plt.show() 

    ok +=8







while (yes<len(k)):   



    plt.figure(figsize=(10,10))
    plt.plot(k[yes+0].history['loss'],color='b', label="opt = Adam")
    plt.plot(k[yes+1].history['loss'],color='r', label="opt = SGD")
    plt.plot(k[yes+2].history['loss'],color='g', label="opt = Nadam")
    plt.plot(k[yes+3].history['loss'],color='c', label="opt = RMSprop")
    plt.plot(k[yes+4].history['loss'],color='m', label="opt = AdaBelief")
    plt.plot(k[yes+5].history['loss'],color='k', label="opt = RAdam")
    plt.plot(k[yes+6].history['loss'],color='y', label="opt = AdaBound")
    plt.plot(k[yes+7].history['loss'],color='#A18820', label="opt = Yogi")
    
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show() 

    yes+=8




while (zet<len(k)):   



    plt.figure(figsize=(10,10))
    plt.plot(k[zet+0].history['val_accuracy'],color='b', label="opt = Adam")
    plt.plot(k[zet+1].history['val_accuracy'],color='r', label="opt = SGD")
    plt.plot(k[zet+2].history['val_accuracy'],color='g', label="opt = Nadam")
    plt.plot(k[zet+3].history['val_accuracy'],color='c', label="opt = RMSprop")
    plt.plot(k[zet+4].history['val_accuracy'],color='m', label="opt = AdaBelief")
    plt.plot(k[zet+5].history['val_accuracy'],color='k', label="opt = RAdam")
    plt.plot(k[zet+6].history['val_accuracy'],color='y', label="opt = AdaBound")
    plt.plot(k[zet+7].history['val_accuracy'],color='#A18820', label="opt = Yogi")
    
    plt.xlabel('Epochs')
    plt.ylabel('val_accuracy')
    plt.legend()
    plt.show() 

    zet+=8





while (yep<len(k)):   



    plt.figure(figsize=(10,10))
    plt.plot(k[yep+0].history['val_loss'],color='b', label="opt = Adam")
    plt.plot(k[yep+1].history['val_loss'],color='r', label="opt = SGD")
    plt.plot(k[yep+2].history['val_loss'],color='g', label="opt = Nadam")
    plt.plot(k[yep+3].history['val_loss'],color='c', label="opt = RMSprop")
    plt.plot(k[yep+4].history['val_loss'],color='m', label="opt = AdaBelief")
    plt.plot(k[yep+5].history['val_loss'],color='k', label="opt = RAdam")
    plt.plot(k[yep+6].history['val_loss'],color='y', label="opt = AdaBound")
    plt.plot(k[yep+7].history['val_loss'],color='#A18820', label="opt = Yogi")
    
    plt.xlabel('Epochs')
    plt.ylabel('val_loss')
    plt.legend()
    plt.show() 

    yep+=8